In [1]:
import argparse
import os
import copy
import sys
import time
from datetime import datetime
import torch
import pdb
import random
import numpy as np
from matplotlib import pyplot as plt
from torch import nn, optim
from torch.nn import functional as F
from torch.utils.data import DataLoader
import pickle

# torch.set_default_dtype(torch.float32)
# from equations.PDEs import *
import sys, os
sys.path.append(os.path.join(os.path.dirname("__file__"), '..'))
sys.path.append(os.path.join(os.path.dirname("__file__"), '..', '..'))
sys.path.append(os.path.join(os.path.dirname("__file__"), '..', '..', '..'))
sys.path.append(os.path.join(os.path.dirname("__file__"), '..', '..', '..', '..'))
sys.path.append(os.path.join(os.path.dirname("__file__"), '..', '..', '..', '..', '..'))
from MP_Neural_PDE_Solvers.equations.PDEs import *
from MP_Neural_PDE_Solvers.common.utils import HDF5Dataset, GraphCreator, p
from MP_Neural_PDE_Solvers.experiments.models_gnn import MP_PDE_Solver
from MP_Neural_PDE_Solvers.experiments.models_cnn import BaseCNN
from MP_Neural_PDE_Solvers.experiments.models_fno import FNO1d
from MP_Neural_PDE_Solvers.experiments.train_helper import *
from MP_Neural_PDE_Solvers.experiments.train import test
from plasma.pytorch_net.util import pload, filter_filename, init_args
server_name = os.uname()[1].split('.')[0]

def check_directory() -> None:
    """
    Check if log directory exists within experiments
    """
    if not os.path.exists(f'experiments/log'):
        os.mkdir(f'experiments/log')
    if not os.path.exists(f'models'):
        os.mkdir(f'models')

In [2]:
dirname = "models/"
model_name, uniform_sample = "BaseCNN", 2
model_name, uniform_sample = "BaseCNN", 3
model_name, uniform_sample = "BaseCNN", -1
# model_name, uniform_sample = "FNO", 2
# model_name, uniform_sample = "FNO", 3
model_name, uniform_sample = "FNO", -1
if model_name == "BaseCNN":
    if uniform_sample == 2:
        filename = "GNN_CE_E2_BaseCNN_xresolution100-200_uni2_n3_tw25_unrolling1_server_ampere2_time9282143_new.pt"
    elif uniform_sample == 3:
        filename = "GNN_CE_E2_BaseCNN_xresolution100-200_uni3_n3_tw25_unrolling1_server_ampere2_time9282143_new.pt"
    elif uniform_sample == -1:
        filename = "GNN_CE_E2_BaseCNN_xresolution100-200_uni-1_n3_tw25_unrolling1_server_ampere2_time928230.pt"
elif model_name == "FNO":
    if uniform_sample == 2:
        filename = "GNN_CE_E2_FNO_xresolution100-200_uni2_n3_tw25_unrolling1_server_ampere2_time9282143_new.pt"
    elif uniform_sample == 3:
        filename = "GNN_CE_E2_FNO_xresolution100-200_uni3_n3_tw25_unrolling1_server_ampere2_time9282143_new.pt"
    elif uniform_sample == -1:
        filename = "GNN_CE_E2_FNO_xresolution100-200_uni-1_n3_tw25_unrolling1_server_ampere1_time928233.pt"
device = "cuda:1"

In [3]:
args = init_args({
    "experiment": "E2",
    "model": model_name,
    "base_resolution": [250,100],
    "time_window": 25,
    "uniform_sample": uniform_sample,
    "batch_size": 4,
    "neighbors": 3,
    "parameter_ablation": False,
    "nr_gt_steps": 2,
})

In [4]:
test_dataset = HDF5Dataset(
    'data/CE_test_E2.h5',
    pde="CE",
    mode='test',
    base_resolution=[250, 100],
    super_resolution=[250, 200],
    uniform_sample=uniform_sample,
)
test_loader = DataLoader(test_dataset,
                         batch_size=args.batch_size,
                         shuffle=False,
                         num_workers=0)
pde = CE(device=device)
pde.tmin = 0.0
pde.tmax = 4.0
pde.grid_size = [250, 100]


eq_variables = {}
if not args.parameter_ablation:
    if args.experiment == 'E2':
        print(f'Beta parameter added to the GNN solver')
        eq_variables['beta'] = 0.2
    elif args.experiment == 'E3':
        print(f'Alpha, beta, and gamma parameter added to the GNN solver')
        eq_variables['alpha'] = 3.
        eq_variables['beta'] = 0.4
        eq_variables['gamma'] = 1.
    elif (args.experiment == 'WE3'):
        print('Boundary parameters added to the GNN solver')
        eq_variables['bc_left'] = 1
        eq_variables['bc_right'] = 1

graph_creator = GraphCreator(pde=pde,
                             neighbors=args.neighbors,
                             time_window=args.time_window,
                             t_resolution=args.base_resolution[0],
                             x_resolution=args.base_resolution[1]).to(device)

if args.model == 'GNN':
    model = MP_PDE_Solver(pde=pde,
                          time_window=graph_creator.tw,
                          eq_variables=eq_variables).to(device)
elif args.model == 'BaseCNN':
    model = BaseCNN(pde=pde,
                    time_window=args.time_window).to(device)
elif args.model == 'FNO':
    model = FNO1d(pde=pde,
                  modes=16, width=64, input_size=args.time_window, output_size=args.time_window).to(device)
else:
    raise Exception("Wrong model specified")
model.load_state_dict(torch.load(dirname + filename))
model.to(device)
steps = [t for t in range(graph_creator.tw, 250-graph_creator.tw + 1)]
criterion = torch.nn.MSELoss(reduction="sum")

Beta parameter added to the GNN solver


In [5]:
# steps = [t for t in range(graph_creator.tw, graph_creator.t_res-graph_creator.tw + 1)]  # tw: 25, t_res: 250. range(25, 250-25+1)
#     losses = test_timestep_losses(model=model,
#                                   steps=steps,
#                                   batch_size=args.batch_size,
#                                   loader=loader,
#                                   graph_creator=graph_creator,
#                                   criterion=criterion,
#                                   device=device)

#     # next we test the unrolled losses
#     losses = test_unrolled_losses(model=model,
#                                   steps=steps,
#                                   batch_size=args.batch_size,
#                                   nr_gt_steps=args.nr_gt_steps,
#                                   nx_base_resolution=args.base_resolution[1],
#                                   loader=loader,
#                                   graph_creator=graph_creator,
#                                   criterion=criterion,
#                                   device=device)

In [6]:
# def test_unrolled_losses(model: torch.nn.Module,
#                      steps: list,  # range(25, 250-25+1)
#                      batch_size: int,
#                      nr_gt_steps: int,  # 2
#                      nx_base_resolution: int,
#                      loader: DataLoader,
#                      graph_creator: GraphCreator,
#                      criterion: torch.nn.modules.loss,
#                      device: torch.cuda.device = "cpu") -> torch.Tensor:
"""
Loss for full trajectory unrolling, we report this loss in the paper
Args:
    model (torch.nn.Module): neural network PDE solver
    steps (list): input list of possible starting (time) points
    nr_gt_steps (int): number of numerical input timesteps
    nx_base_resolution (int): spatial resolution of numerical baseline
    loader (DataLoader): dataloader [valid, test]
    graph_creator (GraphCreator): helper object to handle graph data
    criterion (torch.nn.modules.loss): criterion for training
    device (torch.cuda.device): device (cpu/gpu)
Returns:
    torch.Tensor: valid/test losses
"""

model = model
steps = steps
batch_size = args.batch_size
nr_gt_steps = args.nr_gt_steps
nx_base_resolution = args.base_resolution[1]
loader = test_loader
graph_creator = graph_creator
criterion = criterion
device = device

losses = []
losses_base = []
for (u_base, u_super, x, variables) in loader:
    losses_tmp = []
    losses_base_tmp = []
    with torch.no_grad():
        same_steps = [graph_creator.tw * nr_gt_steps] * batch_size  # [50] * batch_size:16
        data, labels = graph_creator.create_data(u_super, same_steps)  # first time: data: from 25:50, label: from 50:75
        if f'{model}' == 'GNN':
            graph = graph_creator.create_graph(data, labels, x, variables, same_steps).to(device)
            pred = model(graph)
            loss = criterion(pred, graph.y) / nx_base_resolution
        else:
            data, labels = data.to(device), labels.to(device)
            pred = model(data)
            loss = criterion(pred, labels) / nx_base_resolution

        losses_tmp.append(loss / batch_size)

        # Unroll trajectory and add losses which are obtained for each unrolling
        for step in list(range(50, 225, 25)):
            same_steps = [step] * batch_size
            _, labels = graph_creator.create_data(u_super, same_steps)
            if f'{model}' == 'GNN':
                graph = graph_creator.create_next_graph(graph, pred, labels, same_steps).to(device)
                pred = model(graph)
                loss = criterion(pred, graph.y) / nx_base_resolution  # pred/graph.y: [640, 25]
            else:
                labels = labels.to(device)
                pred = model(pred)
                loss = criterion(pred, labels) / nx_base_resolution 
            losses_tmp.append(loss / batch_size)  # batch_size = 16

        # # Losses for numerical baseline
        # for step in range(graph_creator.tw * nr_gt_steps, graph_creator.t_res - graph_creator.tw + 1,
        #                   graph_creator.tw):
        #     same_steps = [step] * batch_size
        #     _, labels_super = graph_creator.create_data(u_super, same_steps)
        #     _, labels_base = graph_creator.create_data(u_base, same_steps)
        #     pdb.set_trace()
        #     loss_base = criterion(labels_super, labels_base) / nx_base_resolution
        #     losses_base_tmp.append(loss_base / batch_size)

    losses.append(torch.sum(torch.stack(losses_tmp)))
    losses_base.append(torch.tensor(0., dtype=torch.float32))
    # losses_base.append(torch.sum(torch.stack(losses_base_tmp)))

losses = torch.stack(losses)
losses_base = torch.stack(losses_base)
print(f'Unrolled forward losses {torch.mean(losses)}')
print(f'Unrolled forward base losses {torch.mean(losses_base)}')


> /dfs/user/tailin/src/plasma/design/MP_Neural_PDE_Solvers/common/utils.py(191)__getitem__()
    189             pdb.set_trace()
    190             # Base resolution trajectories (numerical baseline) and equation specific parameters
--> 191             if self.uniform_sample != -1:
    192                 u_base = self.data[f'pde_{self.base_resolution[0]}-{100}'][idx][...,::self.uniform_sample]
    193                 u_super = u_super[...,::self.uniform_sample]



ipdb>  p self.uniform_sample


-1


ipdb>  n


> /dfs/user/tailin/src/plasma/design/MP_Neural_PDE_Solvers/common/utils.py(196)__getitem__()
    194                 pdb.set_trace()
    195             else:
--> 196                 u_base = self.data[self.dataset_base][idx]
    197             variables = {}
    198             variables['alpha'] = self.data['alpha'][idx]



...

> /dfs/user/tailin/src/plasma/design/MP_Neural_PDE_Solvers/common/utils.py(191)__getitem__()
    189             pdb.set_trace()
    190             # Base resolution trajectories (numerical baseline) and equation specific parameters
--> 191             if self.uniform_sample != -1:
    192                 u_base = self.data[f'pde_{self.base_resolution[0]}-{100}'][idx][...,::self.uniform_sample]
    193                 u_super = u_super[...,::self.uniform_sample]



ipdb>  n


> /dfs/user/tailin/src/plasma/design/MP_Neural_PDE_Solvers/common/utils.py(196)__getitem__()
    194                 pdb.set_trace()
    195             else:
--> 196                 u_base = self.data[self.dataset_base][idx]
    197             variables = {}
    198             variables['alpha'] = self.data['alpha'][idx]



ipdb>  q


BdbQuit: 